In [ ]:
%load_ext autoreload
%autoreload 2
from OSMPythonTools.api import Api
from OSMPythonTools.nominatim import Nominatim
import numpy as np
import pandas as pd
import gudhi as gd
#from pylab import *
from matplotlib import pyplot as plt
import scipy
import scipy.sparse
import sklearn
import sklearn.metrics
import scipy.spatial
import scipy.stats
import sklearn.decomposition
import seaborn as sns
from scipy import stats
import ot
import os
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
import json
from scipy import optimize
import plotly.express as px
from multiprocess import *
from NIPHplot import *
from NIPHSampleGeneration import *
from NIPHomology import *

nominatim = Nominatim()

In [ ]:
# striped = False
# num_samples = 100
# scaling_center = 150
# min_persistance_Barcodes = 0.000000035
# pts_per_sphere = 150
# total_Barcodes = []
# pts_per_sphere= 100
# max_edge_length_Barcodes = 80.
# pdf_array = np.array([scipy.stats.norm.pdf(x,0,1) for x in np.linspace(-2,2,50)])


In [ ]:
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()
areaId = nominatim.query('Tübingen, Germany').areaId()
query = overpassQueryBuilder(area=areaId, elementType='way', selector='"highway"="residential"')
sections_residential_tub = []
for node1 in overpass.query(query, timeout = 80).elements():
    cur_sec = []
    for node in node1.nodes():
        cur_sec.append([node.lat(),node.lon()])
    for i in range(len(cur_sec)-1):
        sections_residential_tub.append([cur_sec[i],cur_sec[i+1]])
with open("secs_residential_tubingen", "w") as fp:
    json.dump(sections_residential_tub, fp)

areaId = nominatim.query('Manhattan, New York').areaId()
query = overpassQueryBuilder(area=areaId, elementType='way', selector='"highway"="residential"')
sections_residential_NYC = []
for node1 in overpass.query(query, timeout = 80).elements():
    cur_sec = []
    for node in node1.nodes():
        cur_sec.append([node.lat(),node.lon()])
    for i in range(len(cur_sec)-1):
        sections_residential_NYC.append([cur_sec[i],cur_sec[i+1]])
with open("secs_residential_NYC", "w") as fp:
    json.dump(sections_residential_NYC, fp)

query = overpassQueryBuilder(area=areaId, elementType='way', selector='"highway"="primary"')
sections_primary_NYC = []
for node1 in overpass.query(query, timeout = 80).elements():
    cur_sec = []
    for node in node1.nodes():
        cur_sec.append([node.lat(),node.lon()])
    for i in range(len(cur_sec)-1):
        sections_primary_NYC.append([cur_sec[i],cur_sec[i+1]])
with open("secs_primary_NYC", "w") as fp:
    json.dump(sections_primary_NYC, fp)


In [ ]:
sections_primary_NYC = np.array(sections_primary_NYC)
sections_residential_NYC = np.array(sections_residential_NYC)
sections_residential_tub = np.array(sections_residential_tub)

sections_primary_NYC[:,:,1]=sections_primary_NYC[:,:,1]*np.cos(40.8*np.pi/180)
sections_residential_NYC[:,:,1]=sections_residential_NYC[:,:,1]*np.cos(40.8*np.pi/180)
sections_residential_tub[:,:,1] = sections_residential_tub[:,:,1]*np.cos(48.5*np.pi/180)
length_primary_NYC = 0
section_length_primary_NYC = []
for section in sections_primary_NYC:
    length_primary_NYC += np.sqrt((section[0][0] - section[1][0])**2 + (section[0][1] - section[1][1])**2)
    section_length_primary_NYC.append(length_primary_NYC)
print(length_primary_NYC)
length_residential_NYC = 0
section_length_residential_NYC = []
for section in sections_residential_NYC:
    length_residential_NYC += np.sqrt((section[0][0] - section[1][0])**2 + (section[0][1] - section[1][1])**2)
    section_length_residential_NYC.append(length_residential_NYC)
print(length_residential_NYC)

length_residential_tub = 0
section_length_residential_tub = []
for section in sections_residential_tub:
    length_residential_tub += np.sqrt((section[0][0] - section[1][0])**2 + (section[0][1] - section[1][1])**2)
    section_length_residential_tub.append(length_residential_tub)
print(length_residential_tub)

cars_primary_NYC = sample_cars(length_primary_NYC,sections_primary_NYC,section_length_primary_NYC, 15000)
cars_residential_NYC = sample_cars(length_residential_NYC,sections_residential_NYC,section_length_residential_NYC, 15000)
cars_residential_tub = sample_cars(length_residential_tub,sections_residential_tub,section_length_residential_tub, 15000)

np.save("cars_total_residential.npy", cars_residential_NYC)
np.save("cars_total_primary.npy", cars_primary_NYC)
np.save("cars_residential_tubingen.npy", cars_residential_tub)

In [ ]:
cars_residential_NYC = np.load('cars_total_residential.npy')
cars_primary = np.load('cars_total_primary.npy')
cars_residential_tubingen = np.load('cars_residential_tubingen.npy')
cars_residential_NYC = np.array(cars_residential_NYC)
cars_primary = np.array(cars_primary)
cars_residential_tubingen = np.array(cars_residential_tubingen)

In [ ]:
square_cars_residential_NYC= cut_out(cars_residential_NYC,40.74,40.76,-56.02,-55.995)
square_cars_primary_NYC = cut_out(cars_primary,40.74,40.76,-56.02,-55.995)
square_cars_residential_tubingen = cut_out(cars_residential_tubingen,48.50,48.54,5.98,6.02)

In [ ]:
import plotly.graph_objects as go
px.scatter(x=square_cars_residential_tubingen[:,1],y=square_cars_residential_tubingen[:,0])
fig = go.Figure()
fig.add_scatter(x=square_cars_residential_tubingen[:,1],y=square_cars_residential_tubingen[:,0],mode='markers')
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.show()

In [ ]:
print(extract_PCA_direction(cars_residential_NYC))
print(extract_PCA_direction(cars_primary))
print(extract_PCA_direction(cars_residential_tubingen))

In [ ]:
def get_values(max_list):
    return np.concatenate((np.array(max_list), np.array(max_list),[np.array(max_list[0])]), axis=0)

arguments = 2*np.pi-np.linspace(0, 2 * np.pi, 41)

In [ ]:
max_list_square_primary_NYC = get_maxs(square_cars_primary_NYC)
max_list_square_residential_NYC = get_maxs(square_cars_residential_NYC)
max_list_primary_NYC = get_maxs(cars_primary)
max_list_residential_tubingen = get_maxs(cars_residential_tubingen)
max_list_square_residential_tubingen = get_maxs(square_cars_residential_tubingen)
max_list_residential_NYC = get_maxs(cars_residential_NYC)

In [ ]:
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
sns.set_style("whitegrid")
ax.plot(arguments,get_values(max_list_square_primary_NYC),label = "Primary roads Manhattan")
plt.fill_between(arguments,get_values(max_list_square_primary_NYC), alpha=0.2)
ax.plot(arguments, get_values(max_list_square_residential_NYC), label = "Residential roads Manhattan")
plt.fill_between(arguments,get_values(max_list_square_residential_NYC), alpha=0.2)
ax.plot(arguments, get_values(max_list_square_residential_tubingen), label = "Residential roads Tübingen")
plt.fill_between(arguments,get_values(max_list_square_residential_tubingen), alpha=0.2)
ax.set_rmax(2.0)
ax.set_rmin(1)
ax.set_rticks([1, 1.2,1.4,1.6,1.8])
ax.legend()
ax.grid(True)

ax.set_title("Road orientations rectangular part", va='bottom')
plt.savefig("Road_orientations_rect.pdf",bbox_inches='tight')
plt.show()

In [ ]:
fig2, ax2 = plt.subplots(subplot_kw={'projection': 'polar'})
sns.set_style("whitegrid")
ax2.plot(arguments,get_values(max_list_primary_NYC),label = "Primary roads Manhattan")
plt.fill_between(arguments,get_values(max_list_primary_NYC), alpha=0.2)
ax2.plot(arguments, get_values(max_list_residential_NYC), label = "Residential roads Manhattan")
plt.fill_between(arguments,get_values(max_list_residential_NYC), alpha=0.2)
ax2.plot(arguments, get_values(max_list_residential_tubingen), label = "Residential roads Tübingen")
plt.fill_between(arguments,get_values(max_list_residential_tubingen), alpha=0.2)
ax2.set_rmax(2.0)
ax2.set_rmin(1)
ax2.set_rticks([1, 1.2,1.4,1.6,1.8,2.0])
ax2.legend()
ax2.grid(True)

ax2.set_title("Road orientationst", va='bottom')
plt.savefig("Road_orientations.pdf",bbox_inches='tight')
plt.show()

In [ ]:
eu_list = [["tubingen",48.52],["Göttingen",51.5], ["Cambridge, UK", 52.2],["Meran",46.67],["Södertälje",59.2],["Belfort, France", 47.64],["Namur, Belgium", 50.47],["Delft",52], ["Quedlinburg", 51.8],["Passau", 48.5],["Jena",50.93], ["Legnica", 51.2], ["Flensburg",47],["Görlitz",51.16], ["King's Lynn and West Norfolk",52.75]]
us_list = [["City of Albany, US",42.65],["South Bend",41.68],["Boulder", 40.01], ["Davenport, Iowa",41.5],["Cupertino", 37.32], ["Nampa, Idaho, US",43.57],["Cheyenne, Wyoming, US",41.16],["Wheaton, Illinois, US",41.86], ["Utica, New York, US",43.11], ["Santa Fe, New Mexico, US",35.69], ["Bismarck, North Dakota, US",46.83], ["Coon Rapids, Minnesota, US",45.12],[ "Great Falls, Montana, US",47.5],["Sunrise, Florida, US",26.18],["Rapid City, South Dakota, US", 44.08]]

In [ ]:
#Skip if already downloaded

from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
overpass = Overpass()
import json
for city in eu_list:
    city_string = city[0]
    print(city_string)
    print("------------------------")
    areaId = nominatim.query(city_string).areaId()
    query = overpassQueryBuilder(area=areaId, elementType='way', selector='"highway"="residential"', out='geom')
    sections_residential = []
    for node1 in overpass.query(query, timeout = 80).elements():
        cur_sec = []
        for node in node1.nodes():
            cur_sec.append([node.lat(),node.lon()])
        for i in range(len(cur_sec)-1):
            sections_residential.append([cur_sec[i],cur_sec[i+1]])
    with open("secs_residential_"+city_string, "w") as fp:
        json.dump(sections_residential, fp)

In [ ]:
overpass = Overpass()
import json
for city in us_list:
    city_string = city[0]
    print(city_string)
    print("------------------------")
    areaId = nominatim.query(city_string).areaId()
    query = overpassQueryBuilder(area=areaId, elementType='way', selector='"highway"="residential"', out='geom')
    sections_residential = []
    for node1 in overpass.query(query, timeout = 80).elements():
        cur_sec = []
        for node in node1.nodes():
            cur_sec.append([node.lat(),node.lon()])
        for i in range(len(cur_sec)-1):
            sections_residential.append([cur_sec[i],cur_sec[i+1]])
    with open("secs_residential_"+city_string, "w") as fp:
        json.dump(sections_residential, fp)

In [ ]:
done_list_us_pca = []
us_results_pca = []

In [ ]:
done_list_eu_pca = []
eu_results_pca = []

In [ ]:
for city_data in eu_list:
    name, lat = city_data
    print(name)
    try:
        if name in done_list_eu_pca:
            print(name+" already done")
        else:
            max_var,var_quot = city_extracter_pca(name,lat)
            eu_results_pca.append([max_var,var_quot,name])
            done_list_eu_pca.append(name)
    except:
        print(name+" failed")

In [ ]:
for city_data in us_list:
    name, lat = city_data
    print(name)
    try:
        if name in done_list_us_pca:
            print(name+" already done")
        else:
            max_var,var_quot = city_extracter_pca(name,lat)
            us_results_pca.append([max_var,var_quot,name])
            done_list_us_pca.append(name)
    except:
        print(name+" failed")

In [ ]:
done_list_us = []
us_results = []
done_list_eu = []
eu_results = []

In [ ]:
for city_data in us_list:
    name, lat = city_data
    print(name)
    try:
        if name in done_list_us:
            print(name+" already done")
        else:
            maxs_cur,var_cur = city_extracter(name,lat)
            us_results.append([maxs_cur,var_cur,np.max(maxs_cur),name])
            done_list_us.append(name)
    except:
        print(name+" failed")

In [ ]:
for city_data in eu_list:
    name, lat = city_data
    print(name)
    try:
        if name in done_list_eu:
            print(name+" already done")
        else:
            maxs_cur,var_cur = city_extracter(name,lat)
            eu_results.append([maxs_cur,var_cur,np.max(maxs_cur),name])
            done_list_eu.append(name)
    except:
        print(name+" failed")

In [ ]:
sns.set_style("whitegrid")
eu_only_results_pca = np.array([[result[1-1],result[2-1],result[3-1],'eu'] for result in eu_results_pca])
us_only_results_pca = np.array([[result[1-1],result[2-1],result[3-1],'us'] for result in us_results_pca if result[2]])
total_results = np.concatenate((eu_only_results_pca,us_only_results_pca),axis=0)
df_eu_pca = pd.DataFrame(eu_only_results_pca,columns=['Max variance','Variance quotient','city','region']).astype({"Max variance": float, "Variance quotient": float})
df_us_pca = pd.DataFrame(us_only_results_pca,columns=['Max variance','Variance quotient','city','region']).astype({"Max variance": float, "Variance quotient": float})
#df_total = pd.DataFrame(total_results,columns=['maximum peak','Variance','city','region']).astype({"maximum peak": float, "variance": float})
fig3, ax3 = plt.subplots(figsize=(4.5, 2))
sns.set(font_scale=1)
thresh = 0.15
sns.scatterplot(
    data=df_eu_pca,
    y="Max variance",
    x="Variance quotient",
    ax=ax3,
    color = sns.color_palette()[0],
    label = "Europe"
)
sns.kdeplot(
    data=df_eu_pca,
    y="Max variance",
    x="Variance quotient",
    levels=3,
    fill=True,
    alpha=0.3,
    cut=2,
    ax=ax3,
    color = sns.color_palette()[0],
    thresh = thresh
)
sns.scatterplot(
    data=df_us_pca,
    y="Max variance",
    x="Variance quotient",
    ax=ax3,
    color = sns.color_palette()[3],
    label = "US"
)
sns.kdeplot(
    data=df_us_pca,
    y="Max variance",
    x="Variance quotient",
    levels=3,
    fill=True,
    alpha=0.3,
    cut=2,
    ax=ax3,
    color = sns.color_palette()[3],
    thresh = thresh
)
plt.show()
#ax.set_xlim(0.5,3)
#ax.set_ylim(-0.0005,0.003)
#plt.savefig('Different_Cities_pda_small.pdf',bbox_inches='tight')


In [ ]:
sns.set_style("whitegrid")
eu_only_results = np.array([[result[1],result[2],result[3],'eu'] for result in eu_results])
us_only_results = np.array([[result[1],result[2],result[3],'us'] for result in us_results])
df_eu = pd.DataFrame(eu_only_results,columns=['Variance of mult. shift peaks','Maximum orientation','city','region']).astype({"Maximum orientation": float, "Variance of mult. shift peaks": float})
df_us = pd.DataFrame(us_only_results,columns=['Variance of mult. shift peaks','Maximum orientation','city','region']).astype({"Maximum orientation": float, "Variance of mult. shift peaks": float})
fig4, ax4 = plt.subplots(figsize=(4.5, 2))
sns.set(font_scale=1)
thresh = 0.15
sns.scatterplot(
    data=df_eu,
    y="Maximum orientation",
    x="Variance of mult. shift peaks",
    ax=ax4,
    color = sns.color_palette()[0],
    label = "Europe"
)
sns.kdeplot(
    data=df_eu,
    y="Maximum orientation",
    x="Variance of mult. shift peaks",
    levels=3,
    fill=True,
    alpha=0.3,
    cut=2,
    ax=ax4,
    color = sns.color_palette()[0],
    thresh = thresh
)
sns.scatterplot(
    data=df_us,
    y="Maximum orientation",
    x="Variance of mult. shift peaks",
    ax=ax4,
    color = sns.color_palette()[3],
    label = "US"
)
sns.kdeplot(
    data=df_us,
    y="Maximum orientation",
    x="Variance of mult. shift peaks",
    levels=3,
    fill=True,
    alpha=0.3,
    cut=2,
    ax=ax4,
    color = sns.color_palette()[3],
    thresh = thresh
)
#ax4.set_xlim(-0.0005,0.005)
plt.show()
#plt.savefig('Different_Cities.pdf',bbox_inches='tight')


In [ ]:
alpha_nyc =gd.AlphaComplex(points = cars_residential_NYC)
st_nyc = alpha_nyc.create_simplex_tree()
Barcodes = st_nyc.persistence()
nyc_zero =[]
for barcode in Barcodes:
    if barcode[0] == 0:
        nyc_zero.append(barcode[1][1])
alpha_tub =gd.AlphaComplex(points = cars_residential_tubingen)
st_tub = alpha_tub.create_simplex_tree()
Barcodes = st_tub.persistence()
tub_zero =[]
for barcode in Barcodes:
    if barcode[0] == 0:
        tub_zero.append(barcode[1][1])

In [ ]:
sns.set(font_scale=2)
#sns.set(x_label="Death time")
ax = sns.kdeplot(np.array(tub_zero*2), clip=[0,0.0000000003], bw_method=0.00001,linewidth=5, color =sns.color_palette()[2],label ="Tübingen")
sns.kdeplot(np.array(nyc_zero), clip=[0,0.0000000003],bw_method=0.0002,linewidth=5,linestyle="--", color = sns.color_palette()[1], label ="Manhattan")
ax.set(xlabel="Death time")
ax.legend()
#plt.savefig("0dimPersNYCTub.pdf", bbox_inches = "tight")
plt.show()